# Data Manipulation in Julia
## David Gold
### 10/26/16 - NYC Julia Users Meetup


# About me
* Ph.D. student in Deptartment of Statistics at the University of Washington
* Worked on NullableArrays.jl at the Recurse Center over the summer of 2015

# This Talk: Two Main Threads

* StructuredQueries.jl (summer project at [Julia Labs](http://julia.mit.edu/))
* Tabular data support in Julia more generally
    * How data manipulation libraries are relevant to planned changes to DataFrames.jl

### Main takeaway: we're (slowly) making progress

## By the way

StructuredQueries.jl and all that follows are works in progress and subject to change. There are design choices that need to be made, implementations that need to be cleaned up/tuned, etc.

All that said, there is a release schedule in mind. 

# StructuredQueries.jl (SQ)

### Query representation framework

Goal: Represent the structure of a "query" with a directed acyclical graph (DAG) object
* "query" like in SQL, or a series of manipulations applied to data, as in `dplyr`

Why?
* ~~Everybody else is using DAGs, so we should too~~
* ~~DAG are my initials, and I want to name something in my package after myself~~
* Doing so allows us to decouple a query's representation from its execution
    * Queries can be generic over different backends (in-memory Julia objects, SQL databases)
    * Solve the "column-indexing" problem
    * Kind of solves the "`Nullable` lifting" problem

(More on the column-indexing and lifting problems later)

# What SQ does

Consider the `iris` dataset.

In [1]:
Pkg.checkout("DataFrames", "master") # at your own risk

INFO: Checking out DataFrames master...
INFO: Pulling DataFrames latest master...
INFO: No packages to install, update or remove


In [ ]:
using DataFrames
using DataStreams
using CSV
iris_csv = CSV.Source(joinpath(Pkg.dir("TablesDemo"), "csv/iris.csv"))
iris = Data.stream!(iris_csv, DataFrame(Data.schema(iris_csv)), false) # Thanks, @quinnj

Suppose we want to restrict to rows whose value for `sepal_length` is greater than `7.5`.


## Vector-based row indexing into `DataFrame`s



In [ ]:
iris[iris[:sepal_length] .> 7.5]

## DataFramesMeta.jl


In [ ]:
# using DataFramesMeta ?

# `@query`

SQ provides the `@query` macro with which to describe a query/series of manipulations against a data source, e.g.

In [ ]:
using StructuredQueries
q = @query filter(iris, sepal_length > 7.5)

We can inspect the representation of the original query using `graph`:

In [ ]:
graph(q)

# What do I do with my `Query`? 

# Extend it

One can extend a `Query` object by querying it:

In [ ]:
q2 = @query q |> select(species, petal_width)

One can use such composability with functions, e.g.

In [ ]:
f(q::Query) = @query q |> select(petal_width)
q3 = f(@query filter(iris, sepal_length > 7.5))
graph(q3)


In [ ]:
q4 = f(@query filter(iris, sepal_width < petal_width))
graph(q4)

# `collect` it

Materialize a query as an in-memory Julia object (e.g. another `DataFrame`) using `collect`

In [ ]:
collect(q)

Whoops! StructuredQueries.jl only houses the graph-producing machinery. Collection machinery lives in another package...

...(tentatively titled)

In [ ]:
using Collect

In [ ]:
collect(q)

"Collect.jl", or whatever it will be called, will re-export StructuredQueries.jl -- users will never have to write `using StructuredQueries`. 

SQ also provides a `@collect` macro that behaves the same as `@query` but automatically collects the resultant `Query`:

In [ ]:
@collect iris |> filter(sepal_length > 7.5)

# How does it work? (for `DataFrame`s)

Without getting too much into the weeds, consider the following query

In [ ]:
graph(q)

We transform the data source into an iterator over tuples, (currently via `zip`) and pass this iterator to an internal function that applies the filtering predicate to each row returned by the iterator. If the predicate is satisfied, the function pushes the row to the result `DataFrame`. The filtering predicate itself is stored in the `FilterNode` object.

(Note that passing the iterator through a function barrier circumvents type-inferability difficulties associated with naively indexing into a `DataFrame` by field, (e.g. naively trying to loop over `df[:sepal_length]`).

For SQL database sources, we can transform the graph into appropriate SQL.

See Yeesian Ng's (@yeesian) work at [SQLQuery](https://github.com/yeesian/SQLQuery.jl), in particular [PR #2](https://github.com/yeesian/SQLQuery.jl/pull/2)

# What else can I do?

# Projections / Transformations

In [ ]:
@collect iris |>
    select(
        species,
        twice_petal_width = 2 * petal_width,
        something = digamma(ifelse(sepal_length > 7.0, rand(), 0.0))
    )

# Grouping / Aggregation


In [ ]:
@collect iris |>
    groupby(species, sepal_length > .5) |>
    summarize(res = mean(log(petal_width)))

# Issues/TODOs

* Name resolution
    * Current parsing machinery assumes that all unadorned names inside function calls are "attributes"
    * What if I want to do 
    
    ```julia
    @collect iris |> summarize(sum = reduce(+, sepal_length))
    ```
    ?
    * Conversely, what if `f` is a field of some `df::DataFrame` whose respective column is a vector of functions? Then 
    ```julia
    @collect select(f(a))
    ``` 
    will not work as expected, because names of *called* functions are inherited from enclosing scope.
* Interpolation/parametrized queries
    * `@collect summarize($c * A)`
* Implementation-wise...
    * Joins like all get out (look at https://github.com/jamii/imp !)

# But where are the benchmarks?

\>_>

# ~~Ruining(?)~~ ~~Bettering(?)~~ Revising the Interface

In [ ]:
Pkg.checkout("StructuredQueries", "with")
Pkg.checkout("AbstractTables", "with")
using StructuredQueries

# Revised interface

In [ ]:
tbl = DataFrame(
    A = rand(10),
    B = rand(10)
);

In [ ]:
q = @with tbl(i) do
    filter(i.A > .5)
    select(C = 5 * i.B)
end

`i` denotes a "row token" that represents an arbitrary row of `tbl` in row-wise operations, such as `filter`ing by row, above.

Very much inspired by LINQ/Query.jl. However, tries to maintain conceptual focus of syntax on data as a whole, not just on iterative aspect. 

In [ ]:
graph(q)

Supports a general yet compact in-line/single-verb syntax:

In [ ]:
q = @with tbl(i) filter(i.A > .5)

In [ ]:
graph(q)

In [ ]:
collect(q)

## Join parsing

Associating a "row token" with each data source encourages regime in which there is no one "privileged" data source argument to each manipulation verb, unlike as in the previously shown interface.

In [ ]:
tbl1, tbl2, tbl3, tbl4 = DataFrame(), DataFrame(), DataFrame(), DataFrame();

In [ ]:
q = @with tbl1(i), tbl2(j) filter(
    i.A < j.B, i.C == "foo", i.D == j.D
)

Note the difference between the type of source shown immediately above and that listed in the `show`ing of `Query` objects earlier. 

In [ ]:
graph(q)

Things can get relatively complicated...

In [ ]:
q = @with tbl1(i), tbl2(j), tbl3(k), tbl4(h) do
    filter(i.A > .5, i.B == j.B, baz(j.C) < .5)
    filter(i.C < .5, j.D == "foo", k.D == h.D, digamma(k.E) > .5, h.F == "bar")
    join(i.D == k.D)
    groupby(f(i.A) < .5, k.D, g(j.B * k.C))
end

...

In [ ]:
graph(q)

## Advantages
* Name resolution is more straightforward
* Refer to attributes of a table in different contexts
    * `i.A` is a "row-wise" context
    * something like `$B` could be used as a "column-context"
        * e.g. 
        ```julia
        @with iris() select(avg = mean(:sepal_length))
        ```
    (could make `select` implicit for extra concision)
* Opens up possibilities for neat window function syntaxes, e.g.
   ```julia
   @with iris(i) filter(
       i.sepal_length > mean([ j.sepal_length for j in iris if j.petal_width > i.petal_width ])
   )
   ```
   (i.e. Restrict to observations `i` from `iris` whose value for `sepal_length` is greater than the mean of all values of `sepal_length` over observations `j` from `iris` whose value for `petal_width` is greater than the present observation `i`'s value of `petal_width`.)    
   (Of course, such functionality is not generic.)
* encourages generality of manipulation verbs over number of sources
* Support for more LINQ/Query.jl-like generality over iterators


## Disadvantages
* Additional complexity
    * for the user, e.g. one has to specify row tokens everywhere
    * for the developer (ie., me): need to keep track of maps from tokens to maps from field names to column indices
* Why is the data source name now a function call?
* Abuse of `.` for denoting field reference
    * (This wouldn't be a problem if we had named tuples!)
    * can always use `Base.getfield` or a selector for actual field retrieval 

# Tabular Data Support More Generally

SQ is a data manipulation interface. Julian data scientists may deal with data in `DataFrame`s. So, let's talk about `DataFrame`s. 

(Stage direction: get on soap box.)

We said above that SQ works by iterating over a tuple-producing iterator. Query.jl works similarly. Iteration is fundamental to both implementations. 

So, we need row-wise iteration over the contents of a `DataFrame` to be fast. Which means that we need such iteration to be type-inferable. 

`DataArray`s  `NullableArray`s achieves this via type-inferable `Base.getindex`.

Alternatives:
* Special case small `Union` types
* Write code like the [`broadcast` implementation for `DataArray`s](https://github.com/JuliaStats/DataArrays.jl/blob/master/src/broadcast.jl)...

# Lifting

Okay, if we go with `NullableArray`s, we have to contend with the fact that indexing into a `NullableArray{T}` returns an object of type `Nullable{T}`. 